# **How do we deal with missing values?**

In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer

In [4]:
melbourne_path = '/home/vosti/machine_learning/csvs/melb_data.csv'
df = pd.read_csv(melbourne_path)
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


## **Features and Target**

In [15]:
# Features having values only
prediction_features = df.drop(['Price'], axis=1)
X = prediction_features.select_dtypes(exclude=['object'])
print(X.head())

#Target
y = df.Price
print(y.head())

   Rooms  Distance  Postcode  Bedroom2  Bathroom  Car  Landsize  BuildingArea  \
0      2       2.5    3067.0       2.0       1.0  1.0     202.0           NaN   
1      2       2.5    3067.0       2.0       1.0  0.0     156.0          79.0   
2      3       2.5    3067.0       3.0       2.0  0.0     134.0         150.0   
3      3       2.5    3067.0       3.0       2.0  1.0      94.0           NaN   
4      4       2.5    3067.0       3.0       1.0  2.0     120.0         142.0   

   YearBuilt  Lattitude  Longtitude  Propertycount  
0        NaN   -37.7996    144.9984         4019.0  
1     1900.0   -37.8079    144.9934         4019.0  
2     1900.0   -37.8093    144.9944         4019.0  
3        NaN   -37.7969    144.9969         4019.0  
4     2014.0   -37.8072    144.9941         4019.0  
0    1480000.0
1    1035000.0
2    1465000.0
3     850000.0
4    1600000.0
Name: Price, dtype: float64


In [38]:
#Divide data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Define model
model = RandomForestRegressor()

# Fit data
model.fit(X_train, y_train)

# predict 
prediction = model.predict(X_test)

prediction

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

We can see that we have missing values from the above error. We can resolve this error by:
*  Dropping columns with missing values
*  Imputation
*  Extension to imputation

First let's define a function `score_dataset()` that returns the MAE after solving the missing value problem using all the aove methods.

In [43]:
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    prediction = model.predict(X_valid)
    mae = mean_absolute_error(y_valid, prediction)
    return mae

## Dropping Columns

In [46]:
# Get columns with missing values
# We get a list of columns in the training data set with missing values
cols_with_missing = [col for col in X_train.columns
                    if X_train[col].isnull().any()] 
print("The colums with missing values are:\n",cols_with_missing)

# Drop the columns with missing values in the training and validation features datasets
dropped_X_train = X_train.drop(cols_with_missing, axis=1)
dropped_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("Original columns in training data: \n", X_train.columns)
print("Columns without missing values are: \n",dropped_X_train.columns)

# MAE of dropping columns method
print("MAE for dropping columns method is: \n",
score_dataset(dropped_X_train, dropped_X_valid, y_train, y_valid))

The colums with missing values are:
 ['Car', 'BuildingArea', 'YearBuilt']
Original columns in training data: 
 Index(['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude',
       'Propertycount'],
      dtype='object')
Columns without missing values are: 
 Index(['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Landsize',
       'Lattitude', 'Longtitude', 'Propertycount'],
      dtype='object')
MAE for dropping columns method is: 
 183550.22137772635


## **Imputation**

Imputation is replacing missing values with other values for example the mean.

In [58]:
my_imputer = SimpleImputer()

imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removes column names. To restore column names
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("The MAE using imputation is:", score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

The MAE using imputation is: 178166.46269899711


The MAE value using imputation is lower compared to dropping columns. This implies Imputation performs better than dropping columns in this case.

## **Extension to Imputation**

Here we keep track of imputed values.

In [70]:
# Make a copy of features for training and validation data
copy_X_train = X_train.copy()
copy_X_valid = X_valid.copy()

# Making new columns indicating what will be imputed
for col in cols_with_missing:
    copy_X_train[col + '_was_missing'] = copy_X_train[col].isnull()
    copy_X_valid[col + 'was missing'] = copy_X_valid[col].isnull()
    
copy_X_train.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount,Car_was_missing,BuildingArea_was_missing,YearBuilt_was_missing
12167,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0,False,True,False
6524,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0,False,True,True
8413,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0,False,True,True
2919,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0,False,True,False
6043,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0,False,False,False


In [71]:
# Imputation
imputed_copy_X_train = pd.DataFrame(my_imputer.fit_transform(copy_X_train))
imputed_copy_X_valid = pd.DataFrame(my_imputer.transform(copy_X_valid))

# Imputation removes column names, to restore them:
imputed_copy_X_train.columns = copy_X_train.columns
imputed_copy_X_valid.columns = copy_X_valid.columns

imputed_copy_X_train.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount,Car_was_missing,BuildingArea_was_missing,YearBuilt_was_missing
0,1.0,5.0,3182.0,1.0,1.0,1.0,0.0,153.764119,1940.000000,-37.85984,144.9867,13240.0,0.0,1.0,0.0
1,2.0,8.0,3016.0,2.0,2.0,1.0,193.0,153.764119,1964.839866,-37.85800,144.9005,6380.0,0.0,1.0,1.0
2,3.0,12.6,3020.0,3.0,1.0,1.0,555.0,153.764119,1964.839866,-37.79880,144.8220,3755.0,0.0,1.0,1.0
3,3.0,13.0,3046.0,3.0,1.0,1.0,265.0,153.764119,1995.000000,-37.70830,144.9158,8870.0,0.0,1.0,0.0
4,3.0,13.3,3020.0,3.0,1.0,2.0,673.0,673.000000,1970.000000,-37.76230,144.8272,4217.0,0.0,0.0,0.0


In [73]:
print("The MAE using extension to imputation is: \n", 
      score_dataset(imputed_copy_X_train, imputed_copy_X_valid, y_train, y_valid))

The MAE using extension to imputation is: 
 178927.503183954
